In [4]:
import pandas as pd

df = pd.read_csv("crt_with_model_responses.csv")
MODELS = ["gemma-7b", "llama-3-8b", "mixtral-8x7b", "llama-3-70b"]

In [6]:
df[[f"{model}-correct" for model in MODELS]].mean()

gemma-7b-correct        0.382353
llama-3-8b-correct      0.617647
mixtral-8x7b-correct    0.705882
llama-3-70b-correct     0.882353
dtype: float64

In [7]:
df[[f"{model}-intuitive" for model in MODELS]].mean()

gemma-7b-intuitive        0.441176
llama-3-8b-intuitive      0.176471
mixtral-8x7b-intuitive    0.147059
llama-3-70b-intuitive     0.088235
dtype: float64